**Imports**

In [1]:
import math, random, numpy as np
from collections import Counter 
from grover_v2 import Grover_Algorithm
from oracles_grover import marked_oracle
from qiskit_aer import AerSimulator
from find_theta import find_thetas
from find_theta_v3 import theta_from_Pm

**Parameter Tuning**

Manual Input

In [30]:
n=128
m_initial = 1
lambda_val=6/5
desired_success_rate=0.99
neighbors=10

Calculated Params i.e dependent on manual params

In [31]:
N_actual=n*(n-1)/2 
N=int(2**math.ceil(math.log2(N_actual)))
alpha=-math.log(1-desired_success_rate)/math.log(n) #Can use n or N
k=int(450*alpha*math.log(n)) 
num_qubits=math.ceil(math.log2(N))
mu_actual=int(neighbors*n/2)

In [32]:
print(N_actual)
print("N =", N)
print("Alpha =",alpha)
print("k =",k)
print("num_qubits =", num_qubits)
print("mu=", mu_actual)

8128.0
N = 8192
Alpha = 0.9491223128249606
k = 2072
num_qubits = 13
mu= 640


**Marked States for Current Fake Oracle**

In [42]:
def random_bitstrings(num_bits: int, num_count: int):
    """Generate num_count unique random bitstrings of length num_bits."""
    if num_count > 2**num_bits:
        raise ValueError("Cannot generate more unique bitstrings than possible combinations.")

    seen = set()
    while len(seen) < num_count:
        bitstring = ''.join(random.choice('01') for _ in range(num_bits))
        seen.add(bitstring)
    return seen   

In [43]:
marked_states= random_bitstrings(num_qubits,mu_actual)
print(marked_states)

{'1001101000000', '1010000100011', '0100111100011', '1110010001010', '0101100111010', '0010100100010', '1000100011001', '1001011011111', '1000101011100', '1101010111001', '1100100110000', '0011111001111', '1110000110110', '1101001110011', '1001110100100', '0010000100110', '1011001111101', '0010110011110', '0001111011011', '0000010111100', '0101000010000', '1010001111011', '0011000100111', '0101010010010', '0000110100000', '0111100100101', '1100101010111', '1111001000011', '0001011011011', '1001111100110', '0010011011011', '1000110110111', '1010001100000', '0011000101011', '1011100010111', '0110101111000', '1100010001011', '0000011100011', '0110001101100', '1110100011001', '1111010010100', '0001101011000', '1100001000000', '1110010101110', '1110000110111', '0101111010010', '1010110110000', '0110011110100', '1000010100000', '0100010001010', '0011001100100', '1100110000011', '0011010000011', '1010101000101', '1111110001101', '0100001101011', '0000001100100', '1100001010000', '100101001111

**Algorithm Elements**

In [44]:
oracle = marked_oracle(marked_states, num_qubits)
backend = AerSimulator()

In [45]:
grover = Grover_Algorithm(
    oracle=oracle,
    backend=backend,
    optimization_level=0,  # 0..3; tune if you want
    measure=True
)

**First Stage**

In [46]:
m = m_initial
A = set()

while True:
    # 1) sample all j values first
    js = [random.randint(0, math.ceil(m)-1) for _ in range(k)]
    groups = Counter(js)  # maps j -> count

    # 2) run each group in one backend call
    n_success = 0
    for j, cnt in groups.items():
        hits,new_seen=grover.run_hits_for_j(j=j, shots=cnt, marked=marked_states,seen=A)
        n_success+=hits
        A.update(new_seen)

    # 3) check success condition
    if n_success / k >= 0.2:
        break

    # 4) update m
    m = min(lambda_val * m, int(math.sqrt(N)))
    print(m)

1.2


Printing out updated m and number of successes in first precritical stage

In [47]:
print("n_success =",n_success)
print("Updated m =",m)
print("n_success/k =",n_success/k)

n_success = 689
Updated m = 1.2
n_success/k = 0.3325289575289575


In [48]:
print(grover._tcache)

{0: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x11c276270>, 1: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x11c797930>}


**Second Stage**

In [49]:
Pm=n_success/k
theta=theta_from_Pm(m=math.ceil(m),Pm=Pm,method='bisection')
mu_initial_estimate= N*np.sin(theta)**2
updated_marked=marked_states-A
unique_seen=len(A)
new_oracle=marked_oracle(updated_marked, num_qubits)
grover.update_oracle(new_oracle)

current_mu_estimate=mu_initial_estimate-unique_seen


In [50]:
print("Unique Seen So Far:", unique_seen)
print("Mu Initial Estimate:", mu_initial_estimate)
print("Current Mu Estimate:", current_mu_estimate)

Unique Seen So Far: 462
Mu Initial Estimate: 668.6846446846349
Current Mu Estimate: 206.68464468463492


**Third Stage**

Original

In [ ]:
while unique_seen<mu_initial_estimate*1.3: #|A|>=tau*mu_init condition
    n_success=0
    k=int(max(0.05*current_mu_estimate, 50))
    j=round(0.58728*np.sqrt(N/current_mu_estimate))
    for _ in range(k):
        result=grover.run_j(j=j)
        if result in updated_marked:
            n_success+=1
            updated_marked.remove(result)
            new_oracle = marked_oracle(updated_marked, num_qubits)
            grover.update_oracle(new_oracle) 
            current_mu_estimate-=1
            A.add(result)
            unique_seen+=1
    if n_success==0:
        break #done=true
    p_success=n_success/k
    estimated_theta = np.arcsin(np.sqrt(p_success))/ (2*j + 1)
    mu_b=N*np.sin(estimated_theta)**2
    current_mu_estimate=round(current_mu_estimate * 0.8 + mu_b* 0.2)
    if current_mu_estimate <= 1:
            current_mu_estimate = 1

print(A) 


After some observances modification

In [44]:
B=5

while unique_seen<mu_initial_estimate*1.3: #|A|>=tau*mu_init condition
    n_success=0
    k=int(max(0.05*current_mu_estimate, 50))
    j=round(0.58728*np.sqrt(N/current_mu_estimate))
    ctr=0
    for _ in range(k):
        result=grover.run_j(j=j)
        if result in updated_marked:
            n_success+=1
            updated_marked.remove(result)
            current_mu_estimate-=1
            A.add(result)
            unique_seen+=1
            ctr+=1
            if ctr==B:
                 new_oracle = marked_oracle(updated_marked, num_qubits)
                 grover.update_oracle(new_oracle) 
                 ctr=0
                 
    if n_success==0:
        break #done=true
    p_success=n_success/k
    estimated_theta = np.arcsin(np.sqrt(p_success))/ (2*j + 1)
    mu_b=N*np.sin(estimated_theta)**2
    current_mu_estimate=round(current_mu_estimate * 0.8 + mu_b* 0.2)
    if current_mu_estimate <= 1:
            current_mu_estimate = 1

**Success Determination**

In [45]:
matching = 0
for i in A:
    if i in marked_states:
        matching+=1
    
achieved_success_rate=matching/len(marked_states)
print(f"Achieved Success Rate: {achieved_success_rate*100}%")

Achieved Success Rate: 100.0%
